---
---

<center><h1>Spark ML</h1></center>
    
    
---

---

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, HashingTF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql import Row

In [2]:
spark = SparkSession.builder \
    .appName("Python Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/02 13:58:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/02 13:58:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/02 13:58:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

23/08/02 13:58:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# Create SparkContext
sc = SparkContext.getOrCreate()
sc.setLogLevel("ERROR")

# Create SQLContext
sqlContext = SQLContext(sc)


/Users/kshitijchandra/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
# Load the training data
dataframe = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('train.csv')



In [6]:
dataframe.show()

+---+-----+--------------------+
| id|label|               tweet|
+---+-----+--------------------+
|  1|    0| @user when a fat...|
|  2|    0|@user @user thank...|
|  3|    0|  bihday your maj...|
|  4|    0|#model   i love u...|
|  5|    0| factsguide: soci...|
|  6|    0|[2/2] huge fan fa...|
|  7|    0| @user camping to...|
|  8|    0|the next school y...|
|  9|    0|we won!!! love th...|
| 10|    0| @user @user welc...|
| 11|    0| â #ireland con...|
| 12|    0|we are so selfish...|
| 13|    0|i get to see my d...|
| 14|    1|@user #cnn calls ...|
| 15|    1|no comment!  in #...|
| 16|    0|ouch...junior is ...|
| 17|    0|i am thankful for...|
| 18|    1|retweet if you ag...|
| 19|    0|its #friday! ð...|
| 20|    0|as we all know, e...|
+---+-----+--------------------+
only showing top 20 rows



In [7]:
# Perform basic analysis on the data
print(f'Number of rows in the dataset: {dataframe.count()}')
print('Percentage of each class:')
dataframe.groupBy('label').count().show()

Number of rows in the dataset: 31962
Percentage of each class:
+-----+-----+
|label|count|
+-----+-----+
|    1| 2242|
|    0|29720|
+-----+-----+



In [8]:
# Tokenizer and HashingTF are transformers
tokenizer = Tokenizer(inputCol="tweet", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

# LogisticRegression is an estimator
lr = LogisticRegression(maxIter=10, regParam=0.001)

# Create Pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training data.
model = pipeline.fit(dataframe)


In [9]:
# Create StreamingContext with batch interval of 3 seconds
ssc = StreamingContext(sc, 3)

# Specify the host and port to read data from
lines = ssc.socketTextStream("localhost", 9999)

words = lines.flatMap(lambda line: line.split(" "))


/Users/kshitijchandra/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [10]:
def process_rdd(time, rdd):
    print("========= %s =========" % str(time))
    try:
        # Convert RDD to DataFrame
        row_rdd = rdd.map(lambda word: Row(tweet=word))
        words_data_frame = spark.createDataFrame(row_rdd)

        # Make predictions
        prediction = model.transform(words_data_frame)

        # Show predictions
        prediction.show()

    except Exception as e:
        pass  # You could print the exception here if needed

# Apply this function to each RDD in the DStream
words.foreachRDD(process_rdd)

ssc.start()
ssc.awaitTermination()

========= 2023-08-02 13:59:12 =========
========= 2023-08-02 13:59:15 =========
========= 2023-08-02 13:59:18 =========
========= 2023-08-02 13:59:21 =========
========= 2023-08-02 13:59:24 =========
========= 2023-08-02 13:59:27 =========
========= 2023-08-02 13:59:30 =========


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/kshitijchandra/.pyenv/versions/3.9.16/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/kshitijchandra/.pyenv/versions/3.9.16/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/kshitijchandra/.pyenv/versions/3.9.16/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

---
<center><h1>END</h1></center>

---
